In [1]:
from autoencoders import Conv1dAutoEncoder, LSTMAutoEncoder, TickerDataModule, MLPAutoEncoder

from typing import Any
import pandas as pd
import numpy as np
from math import floor
import torch
import json

from tqdm.notebook import tqdm

In [2]:
with open('config/config.json', 'r') as file:
    config = json.load(file)
config

{'start_date': '2018-10-23',
 'end_date': '2022-10-23',
 'riskless_rate': 0.03,
 'n_clusters': 11,
 'n_stock_portfolio': 2,
 'random_state': 42,
 'ticker_companies': 'data/sp500_tickers.csv',
 'tickers_sectors_path': 'data/ticker_sectors.csv',
 'ticker_data_close': 'data/ticker_data_Close.csv',
 'ticker_data_sp500': 'data/ticker_data_SP500.csv',
 'ticker_data_volume': 'data/ticker_data_Volume.csv',
 'ticker_data_preprocessed': 'data/ticker_data_preprocessed.csv',
 'features_path': 'data/features.csv',
 'nn_mlp_checkpoint': 'autoencoders/lightning_logs/mlp/version_0/checkpoints/epoch=149-step=24749.ckpt',
 'nn_lstm_checkpoint': 'autoencoders/lightning_logs/lstm/version_0/checkpoints/epoch=99-step=14699.ckpt',
 'nn_conv_checkpoint': 'autoencoders/lightning_logs/cae/version_0/checkpoints/epoch=99-step=16499.ckpt',
 'nn_mlp_data': 'data/ticker_data_mlp.csv',
 'nn_lstm_data': 'data/ticker_data_lstm.csv',
 'nn_conv_data': 'data/ticker_data_cae.csv',
 'metrics_path': 'results/metrics_table.cs

In [3]:
df = pd.read_csv(config['ticker_data_preprocessed'], index_col=[0])
df.drop(columns=['sector'], axis=1, inplace=True)

In [5]:
model_mlp = MLPAutoEncoder.load_from_checkpoint(config['nn_mlp_checkpoint'],
                                            in_features=100,
                                            latent_features=18)
model_lstm = LSTMAutoEncoder.load_from_checkpoint(config['nn_lstm_checkpoint'],
                                            seq_len=100,
                                            n_features=1, 
                                            embedding_dim=18)
model_cae = Conv1dAutoEncoder.load_from_checkpoint(config['nn_conv_checkpoint'],
                                            in_channels=1,
                                            n_latent_features=18, 
                                            seq_len=100)
model_mlp.eval()
model_lstm.eval()
model_cae.eval();

1


In [6]:
mlp_encoded = np.zeros((df.shape[0], 18))
lstm_encoded = np.zeros((df.shape[0], 18))
cae_encoded = np.zeros((df.shape[0], 18))

for i, name_ticker in tqdm(enumerate(np.unique(df.index))):
    ts_name = df[df.index == name_ticker].values
    ts_name = ts_name.flatten()
    seq_len = ts_name.shape[0]
    fl_1 = floor(seq_len / 100)
    sample_1 = ts_name[:100 * fl_1].reshape(fl_1, 1, 100)
    fl_2 = floor(seq_len / 100)
    sample_2 = ts_name[:100 * fl_2].reshape(fl_2, 1, 100)
    
    mlp_sample = model_mlp.predict_step(torch.tensor(sample_1).float()).detach().numpy()
    cae_sample = model_cae.predict_step(torch.tensor(sample_1).float()).squeeze().detach().numpy()
    lstm_sample = model_lstm.predict_step(torch.tensor(sample_2).float()).detach().numpy()
    
    mlp_emb = mlp_sample.mean(axis=0)
    cae_emb = cae_sample.mean(axis=0)
    lstm_emb = lstm_sample.mean(axis=0)
    
    mlp_encoded[i, :] = mlp_emb
    cae_encoded[i, :] = cae_emb
    lstm_encoded[i, :] = lstm_emb
        

0it [00:00, ?it/s]

In [7]:
df_mlp = pd.DataFrame(mlp_encoded, index=np.unique(df.index))
df_cae = pd.DataFrame(cae_encoded, index=np.unique(df.index))
df_lstm = pd.DataFrame(lstm_encoded, index=np.unique(df.index))

In [8]:
df_mlp.to_csv(config['nn_mlp_data'])
df_cae.to_csv(config['nn_conv_data'])
df_lstm.to_csv(config['nn_lstm_checkpoint'])